이제부터는 신경망을 구성하는 층 각각을 클래스 하나로 구현한다. 먼저 활성화 함수인 ReLU와 Sigmoid 계층을 구현해볼 것이다.

## ReLU 계층
ReLU 함수의 수식은 다음과 같다.
<img src="img/deep_learning_images/e_5.7.png" width=150 height=150>

위 수식에서 x에 대한 y의 미분은
<img src="img/deep_learning_images/e_5.8.png" width=150 height=150>

이를 통해 알 수 있는 점은 순전파의 입력인 x가 0보다 크면 역전파는 이전 노드의 값을 그대로 다음 노드로 흘린다.(1이 곱해지는 것이니까 값에 변동이 없음)  
반면에 순전파 때 x가 0이하일 경우 역전파 때는 다음 노드로 신호를 보내지 않는다.(0을 보냄) 이를 그림으로 나타내보면,
<img src="img/deep_learning_images/fig_5-18.png" width=512 height=512>

In [2]:
import numpy as np

In [1]:
class Relu:
    def __init__(self):
        self.mask = None # True와 False로 구성된 넘파이 배열
        
    def forward(self, x):
        self.mask = (x <= 0) # 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 0보다 큰 원소는 False로.
        out = x.copy()
        out[self.mask] = 0 # True인 경우 out의 원소는 0이 된다.
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [3]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [4]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


In [5]:
test_relu_layer = Relu()
print(test_relu_layer.forward(x))

[[1. 0.]
 [0. 3.]]


In [6]:
mask = (x <= 0)
print(mask)

x[mask] = 0
print(x)

[[False  True]
 [ True False]]
[[1. 0.]
 [0. 3.]]


## Sigmoid 계층
<img src="img/deep_learning_images/e_5.9.png" width=180 height=180>

<img src="img/deep_learning_images/fig_5-19.png" width=448 height=448>
<img src="img/deep_learning_images/sigmoid_node.png" width=416 height=416>

먼저 나눗셈 노드에 대한 미분을 하면,
<img src="img/deep_learning_images/e_5.10.png" width=120 height=120>
식에서 알 수 있듯 이전 노드에서 흘러온 값에 $-y^2$(순전파의 출력을 제곱하고, 마이너스를 붙인 값)을 곱해서 다음 노드로 전달한다.

exp 노드는 $y=exp(x)$연산인데 미분 결과는 다음과 같다.
<img src="img/deep_learning_images/e_5.11.png" width=150 height=150>

즉, 이전 노드의 값에 순전파 때의 출력을 곱해(여기서는 $exp(-x)$) 다음 노드로 전달.

<img src="img/deep_learning_images/fig_5-19(3).png" width=448 height=448>
<img src="img/deep_learning_images/fig_5-20.png" width=448 height=448>

여기서 역전파 값을 한 번 더 정리하면,
<img src="img/deep_learning_images/e_5.12.png" width=448 height=448>
<img src="img/deep_learning_images/sigmoid_formula_reform.png" width=256 height=256>

정리한 식의 마지막에서 볼 수 있듯 순전파의 출력값만으로 역전파를 계산할 수 있음을 알 수 있다.

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx